In [51]:
class CurrentBoard:
  
  
  def __init__(self,setup_of_board = ['wwww','wwww','wwww','    ','    ','bbbb', 'bbbb', 'bbbb'], newEvaluation = None):
      self.board = setup_of_board
      if newEvaluation != None:
        self.state_of_play = newEvaluation
      self.state, self.evaluation  = self.state_of_play(self.board)

  def new_board(self,new_setup):
    self.board = new_setup
    self.state,self.evaluation = self.state_of_play(self.board)


  def display_board(self):
    print("-----------------")
    for row in range(8):
      c = self.board[row]
      if (row % 2) == 0:
        print("| |{}| |{}| |{}| |{}|".format(c[0], c[1],c[2],c[3]))
      else: 
        print("|{}| |{}| |{}| |{}| |".format(c[0], c[1],c[2],c[3]))
      print("-----------------")
    



  def adjacent_squares(self, position, inc = 999):
    # inc = 1 for whites, inc = -1 for black, inc  =0 for kings
    # return a list of all possible positions reachable in 1 move from position

    if inc  > 10:
      return self.adjacent_squares(position,inc = self.directionAt(position))
    if inc == 0:
        return self.adjacent_squares(position, inc = -1) + self.adjacent_squares(position,inc = 1)

    all_moves = []
    start_row = position[0]
    start_i = position[1]

    new_row = start_row+inc
    if (new_row>=0) and (new_row < 8):
      all_moves.append((new_row,start_i))
      if (start_row % 2) == 0:
        if (start_i<3):
          all_moves.append((new_row, start_i+1))
      else:
        if (start_i >0):
          all_moves.append((new_row, start_i-1))       

    return all_moves

  def playingAs(self,piece):
    if piece == 'W':
      return 'w'
    if piece == "B":
      return 'b'
    return piece


  def all_possible_moves_for(self,playing_colour): # all moves for "w" or "b"
    non_kill_boards = []
    kill_boards = []
    found_kill = False
    for row in range(8):
      for i in range(4):
        if self.playingAs(self.get_piece_at((row,i))) == playing_colour:
          new_non_kills, new_kills = self.construct_boards_for((row,i),found_kill)
          if len(new_kills) > 0:
            found_kill = True
            kill_boards += new_kills
          else:
            non_kill_boards += new_non_kills
    if found_kill:
      return kill_boards
    else:
      return non_kill_boards



  def get_piece_at(self, position):
    if position != None:
      return self.board[position[0]][position[1]]
    else:
      return None

  def row_for(self,position):
    return position[0]

  def i_for(self,position):
    return position[1]


  def jump_to(self, from_position, over_position):
    row_dir = self.row_for(over_position) - self.row_for(from_position)
    new_row = self.row_for(over_position) + row_dir
    if (new_row <0) or (new_row>7):
      return None
    if (new_row % 2) == 0:
      if (self.i_for(from_position) == self.i_for(over_position)):
        new_i = self.i_for(from_position) - 1
      else:
        new_i = self.i_for(from_position)  +1
    else:

      if (self.i_for(from_position) == self.i_for(over_position)):
        new_i = self.i_for(from_position) + 1
      else:
        new_i = self.i_for(from_position)  -1   
    if (new_i >=0) and (new_i<4):
      return (new_row,new_i)

    
  def all_kills(self,start_position, current_position,kill_list):


    all_kills_from_here = self.get_adjacent_kills(current_position, kill_list)
    all_completed_moves = []
    if (all_kills_from_here == []):
      if (kill_list == []):
        return []
      else:
        return [self.board_for(start_position, current_position,kill_list)]
    
    for kill in all_kills_from_here:
      all_completed_moves += self.all_kills(start_position,self.jump_to(current_position, kill), kill_list + [kill])

    return all_completed_moves

  def king_me(self,piece):
    if piece == 'w':
      return 'W'
    if piece == 'b':
      return 'B'
    return piece
  
  def board_for(self, from_position, to_position, kill_list):
    board_def = []
    remove_list = kill_list +[from_position]
  
    piece = self.get_piece_at(from_position)
    if (self.row_for(to_position) %7 ) == 0:
      piece = self.king_me(piece)
    for row_index in range(8):
      row = self.board[row_index]
      for empty_square in remove_list:
        if row_index == self.row_for(empty_square):
          row =  self.replace_at(row, self.i_for(empty_square) ,' ' ) 
      if row_index == self.row_for(to_position):

        row = self.replace_at(row, self.i_for(to_position) , piece )
      
      board_def.append(row)
 
    return CurrentBoard(board_def,newEvaluation=self.state_of_play)



  def move_piece_to(self, from_position, to_position):

    return self.board_for(from_position,to_position,[from_position])


  def get_adjacent_kills(self,position,kill_list):
      piece  = self.get_piece_at(position)
      kills = []
      all_adjacent_squares = self.adjacent_squares(position)
      for square in all_adjacent_squares:
        if self.is_opponent_of(piece,self.get_piece_at(square)):
          if self.is_not_already_killed(square,kill_list) and self.get_piece_at( self.jump_to(position,square) ) == ' ':
            kills.append(square)
      return kills

  def is_not_already_killed(self,square, list_of_positions):
    return not ( square in list_of_positions)



  def is_opponent_of(self,piece1,piece2):
    return  (self.playingAs(piece1) != self.playingAs(piece2)) and (piece1 != ' ') and (piece2 != ' ')




  def replace_at(self,row_as_string, index, character):
    return row_as_string[:index] + character + row_as_string[index+1:]

  def construct_boards_for(self,  from_position , found_kill):
    non_kill_boards = []
    kill_boards = []

    kill_boards += self.all_kills(from_position,from_position,[])
    if not found_kill  and len(kill_boards)==0:
      to_position_list = self.adjacent_squares(from_position)
      for new_position in to_position_list:
        if self.get_piece_at(new_position) == " ":
          non_kill_boards.append(self.move_piece_to(from_position, new_position))

    return non_kill_boards, kill_boards


  def directionAt(self,position):
    piece = self.get_piece_at(position)
    color = self.playingAs(piece)
    inc  = 0 
    if piece == color:
      if piece == 'w':
        inc = 1
      else:
        inc = -1
    return inc



    
  






  



In [52]:
class Search_Tree_Node:
  def __init__(self, game , playing_char, ply = 0, max_ply = 6):
    self.children =[]
    self.ply_depth = ply
    self.max_ply = max_ply
    self.value_is_assigned = False
    self.value = 999
    self.board_for_node = game
    self.board_eval = game.evaluation if playing_char == "w" else -game.evaluation
    self.playing = playing_char
    if self.board_for_node.state == "U":
      self.generate_children()
    else:
      # game is won by either black or white
      nodePlayingSameasAI = (self.ply_depth %2 == 0)

      if nodePlayingSameasAI:
        if self.playing == self.board_for_node.state:
          self.value = 1000
        else:
          self.value = -1000

      else:
        if self.playing == self.board_for_node.state:
          self.value = -1000
        else:
          self.value = 1000

      self.value_is_assigned = True
      



  def max_min_value(self):
    # if no value is assigned we will assign the max(or min) of the value of the children

    if  self.value_is_assigned:
        return self.value

    self.children = sorted(self.children, key= lambda x: x.max_min_value() )
    self.value_is_assigned = True
    if (self.ply_depth % 2) == 0:
      self.value = self.children[-1].value
    else:
      self.value = self.children[0].value
    return self.value


  def other_char(self,current):
    if current == "w":
      return "b"
    if current == "b":
      return "w"
    return " "

  def generate_children(self):
    if self.max_ply > self.ply_depth:
      for next_move in self.board_for_node.all_possible_moves_for(self.playing):
        self.children.append(Search_Tree_Node(next_move, self.other_char(self.playing), self.ply_depth + 1))
      if len(self.children) == 0:
        self.value_is_assigned = True
        if (self.ply_depth % 2) == 0:
          self.value = -1000000  
        else:
          self.value = 1000000
    
    
    else:
      self.value = self.board_eval
      self.value_is_assigned = True


In [53]:
def versusBots(evalf1, evalf2):
  max_ply = 6
  V1MoveNext = False;
  V1isPlaying ='b'
  print("First Eval function is playing " + V1isPlaying)
  V2isPlaying = 'w'

  if V1MoveNext:
    currentBoard = CurrentBoard(newEvaluation=evalf1)
  else:
    currentBoard = CurrentBoard(newEvaluation=evalf2)

  currentGameOver = False


  while not currentGameOver:
    if V1MoveNext:
      st = Search_Tree_Node(currentBoard,V1isPlaying, max_ply=max_ply)
      st.max_min_value()
      if len(st.children) == 0:
        currentGameOver = True
        currentBoard.state = V2isPlaying
      else: 
        currentBoard = CurrentBoard(st.children[-1].board_for_node.board, evalf2)
    

    else:  # V2 is playing next
      st = Search_Tree_Node(currentBoard,V2isPlaying, max_ply=max_ply)
      st.max_min_value()
      if len(st.children) == 0:
        currentGameOver = True
        currentBoard.state = V1isPlaying
      else:
        currentBoard =  CurrentBoard(st.children[-1].board_for_node.board, evalf1)
    
    
    currentGameOver = currentBoard.state != "U"
    
    print("The evaluation of the boards below is " + str(currentBoard.evaluation))
    currentBoard.display_board()
    V1MoveNext = not V1MoveNext
    if max_ply<8:
      count = 0
      for r in currentBoard.board:
        count+= r.count(' ')

      if count > 20:
        max_ply = 15;


  print("The winner is " +str(currentBoard.state))


In [54]:

def state_of_play(boardAsString):

  whites = 0
  blacks = 0
  for row in boardAsString:
    whites += row.count('w') + 2* row.count('W')
    blacks += row.count('b') + 2* row.count('B')
  
  evaluation = whites - blacks
  if whites == 0 :
    return 'b', evaluation
  if blacks == 0:
    return 'w', evaluation
  return 'U', evaluation




def state_of_playV2(boardAsString):
    pieceValue = 1
    kingMultiplier = 2.5


    whites = 0
    blacks = 0

    for row in boardAsString:
      whites += row.count('w')+ kingMultiplier* row.count('W')
      blacks += row.count('b')+ kingMultiplier*  row.count('B')
    
    evaluation = whites - blacks
    if whites == 0 :
      return 'b',evaluation
    if blacks == 0:
      return 'w',evaluation
    return 'U',evaluation



For V3 I am going to increase the value of the King further to see if placing more weight on the King pieces beats V2.  

In [55]:
#v3 - increase the value of the kings in the evaluation function.
def state_of_playV3(boardAsString):
    pieceValue = 1
    #multiplier increased to 5 rather than 2.5
    kingMultiplier = 5


    whites = 0
    blacks = 0

    for row in boardAsString:
      whites += row.count('w')+ kingMultiplier* row.count('W')
      blacks += row.count('b')+ kingMultiplier*  row.count('B')
    
    evaluation = whites - blacks
    if whites == 0 :
      return 'b',evaluation
    if blacks == 0:
      return 'w',evaluation
    return 'U',evaluation

In [8]:
versusBots(state_of_playV2 ,state_of_playV3)

First Eval function is playing b
The evaluation of the boards below is 0.0
-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| |w| |w| |w| | |
-----------------
| | | | | | |w| |
-----------------
| | | | | | | | |
-----------------
|b| |b| |b| |b| |
-----------------
| |b| |b| |b| |b|
-----------------
|b| |b| |b| |b| |
-----------------
The evaluation of the boards below is 0
-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| |w| |w| |w| | |
-----------------
| | | | | | |w| |
-----------------
| | | | | |b| | |
-----------------
|b| |b| |b| | | |
-----------------
| |b| |b| |b| |b|
-----------------
|b| |b| |b| |b| |
-----------------
The evaluation of the boards below is 0.0
-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| |w| |w| |w| | |
-----------------
| | | | | | | | |
-----------------
| | | | | |b| |w|
-----------------
|b| |b| |b| | | |
---------------

KeyboardInterrupt: ignored

In [9]:
versusBots(state_of_playV3 ,state_of_playV2)

First Eval function is playing b
The evaluation of the boards below is 0
-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| |w| |w| |w| | |
-----------------
| | | | | | |w| |
-----------------
| | | | | | | | |
-----------------
|b| |b| |b| |b| |
-----------------
| |b| |b| |b| |b|
-----------------
|b| |b| |b| |b| |
-----------------
The evaluation of the boards below is 0.0
-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| |w| |w| |w| | |
-----------------
| | | | | | |w| |
-----------------
| | | | | |b| | |
-----------------
|b| |b| |b| | | |
-----------------
| |b| |b| |b| |b|
-----------------
|b| |b| |b| |b| |
-----------------
The evaluation of the boards below is 0
-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| |w| |w| |w| | |
-----------------
| | | | | | | | |
-----------------
| | | | | |b| |w|
-----------------
|b| |b| |b| | | |
-----------------

KeyboardInterrupt: ignored

**V2 vs V3:**  
When V3 goes first it becomes an infinite loop as both sides value the king pieces highly and end up just keeping their King pieces safe by moving back and forth on opposite sides of the board. V2 has 3 kings and one regular piece while V3 has only 2 kings and one normal piece so it could be considered that V2 performed better. 

When V2 goes first it again results in an infinite loop but V3 has two Kings while V2 has one king and one normal piece so it could be considered that V3  was in a stronger position. 

Overall the two games resulted in an infinite loop and who was in a stronger position depended on who went first. It is therefore difficult to determine which evaluation function is stronger.




In [12]:
#v4 - add board position to the value of the pieces, where being a position closer to becoming kinged is worth more than being on your own side. Values are increased in the decimal points. Note this only applies to the normal pieces ('w/b' as the position of the kinged pieces doesn't matter, as they are already kinged. ) 
def state_of_playV4(boardAsString):
    pieceValue = 1
    #multiplier kept at 5 for V4 - v5 will have the same function but with the multiplier at 2.5 to see which is better. 
    kingMultiplier = 5


    whites = 0
    blacks = 0

    for row in boardAsString:
      whites += row.count('w')+ kingMultiplier* row.count('W') + boardAsString.index(row) * row.count('w')/100
      blacks += row.count('b')+ kingMultiplier*  row.count('B') - (7-boardAsString.index(row)) + row.count('b')/100
    
    evaluation = whites - blacks
    if whites == 0 :
      return 'b',evaluation
    if blacks == 0:
      return 'w',evaluation
    return 'U',evaluation

In [13]:
#v5 - same as v4 but the kingg multiplier is lower
def state_of_playV5(boardAsString):
    pieceValue = 1
    kingMultiplier = 2.5


    whites = 0
    blacks = 0

    for row in boardAsString:
      whites += row.count('w')+ kingMultiplier* row.count('W') + boardAsString.index(row) * row.count('w')/100
      blacks += row.count('b')+ kingMultiplier*  row.count('B') - (7-boardAsString.index(row)) + row.count('b')/100
    
    evaluation = whites - blacks
    if whites == 0 :
      return 'b',evaluation
    if blacks == 0:
      return 'w',evaluation
    return 'U',evaluation

In [14]:
#v4 vs v5 to see which king multiplier is better 
versusBots(state_of_playV4 ,state_of_playV5)

First Eval function is playing b
The evaluation of the boards below is 31.970000000000002
-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| |w| |w| | | |w|
-----------------
| | | | | | |w| |
-----------------
| | | | | | | | |
-----------------
|b| |b| |b| |b| |
-----------------
| |b| |b| |b| |b|
-----------------
|b| |b| |b| |b| |
-----------------
The evaluation of the boards below is 29.97
-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| |w| |w| | | |w|
-----------------
| | | | | | |w| |
-----------------
| | | | | |b| | |
-----------------
|b| |b| | | |b| |
-----------------
| |b| |b| |b| |b|
-----------------
|b| |b| |b| |b| |
-----------------
The evaluation of the boards below is 32.0
-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| |w| |w| | | |w|
-----------------
| | | | | | | | |
-----------------
| | | | | | | | |
-----------------
|b| |b| |w| |

In [16]:
#v4 vs v5 to see which king multiplier is better 
versusBots(state_of_playV5 ,state_of_playV4)

First Eval function is playing b
The evaluation of the boards below is 31.970000000000002
-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| |w| |w| | | |w|
-----------------
| | | | | | |w| |
-----------------
| | | | | | | | |
-----------------
|b| |b| |b| |b| |
-----------------
| |b| |b| |b| |b|
-----------------
|b| |b| |b| |b| |
-----------------
The evaluation of the boards below is 29.97
-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| |w| |w| | | |w|
-----------------
| | | | | | |w| |
-----------------
| | | | | |b| | |
-----------------
|b| |b| | | |b| |
-----------------
| |b| |b| |b| |b|
-----------------
|b| |b| |b| |b| |
-----------------
The evaluation of the boards below is 32.0
-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| |w| |w| | | |w|
-----------------
| | | | | | | | |
-----------------
| | | | | | | | |
-----------------
|b| |b| |w| |

**V4 vs V5:** 
When V4 went first V5 won, and when V5 went first V4 won so it is difficult to determine which King Multiplier leads to a better evaluation function. V5 won with 6 pieces left (4 normal and 2 Kings), while V4 won with 6 pieces (5 normal and 1 King), indicating that V5 is sightly stronger. I will now test V5 against V3 to see if it is a more effective agent. 

In [17]:
#v3 vs v5 to see which king multiplier is better 
versusBots(state_of_playV5 ,state_of_playV3)

First Eval function is playing b
The evaluation of the boards below is 31.970000000000002
-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| |w| |w| |w| | |
-----------------
| | | | | | |w| |
-----------------
| | | | | | | | |
-----------------
|b| |b| |b| |b| |
-----------------
| |b| |b| |b| |b|
-----------------
|b| |b| |b| |b| |
-----------------
The evaluation of the boards below is 0
-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| |w| |w| |w| | |
-----------------
| | | | | | |w| |
-----------------
| | | | | |b| | |
-----------------
|b| |b| | | |b| |
-----------------
| |b| |b| |b| |b|
-----------------
|b| |b| |b| |b| |
-----------------
The evaluation of the boards below is 32.0
-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| |w| |w| |w| | |
-----------------
| | | | | | | | |
-----------------
| | | | | | | | |
-----------------
|b| |b| |w| |b| |

In [18]:
#v3 vs v5 to see which king multiplier is better 
versusBots(state_of_playV3 ,state_of_playV5)

First Eval function is playing b
The evaluation of the boards below is 0
-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| |w| |w| | | |w|
-----------------
| | | | | | |w| |
-----------------
| | | | | | | | |
-----------------
|b| |b| |b| |b| |
-----------------
| |b| |b| |b| |b|
-----------------
|b| |b| |b| |b| |
-----------------
The evaluation of the boards below is 29.97
-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| |w| |w| | | |w|
-----------------
| | | | | | |w| |
-----------------
| | | | | |b| | |
-----------------
|b| |b| |b| | | |
-----------------
| |b| |b| |b| |b|
-----------------
|b| |b| |b| |b| |
-----------------
The evaluation of the boards below is 0
-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| | | |w| |
-----------------
| |w| |w| |w| |w|
-----------------
| | | | | | |w| |
-----------------
| | | | | |b| | |
-----------------
|b| |b| |b| | | |
---------------

**V3 vs V5**
V3 wins both games i.e. when starting first and when starting second. So V3 is stringer than V5. Lets check if it is also stroonger than V4. 

In [19]:
#v3 vs v4 
versusBots(state_of_playV3 ,state_of_playV4)

Streaming output truncated to the last 5000 lines.
| | |b| | | |b| |
-----------------
| |b| | | | | | |
-----------------
| | | | | | | | |
-----------------
| | | | | | | | |
-----------------
| | | | | | | | |
-----------------
| |B| | | |W| | |
-----------------
| | | | | | | | |
-----------------
The evaluation of the boards below is 36.97
-----------------
| | | | | | | | |
-----------------
| | |b| | | |b| |
-----------------
| |b| | | | | | |
-----------------
| | | | | | | | |
-----------------
| | | | | | | | |
-----------------
| | | | | | | | |
-----------------
| | | | | |W| | |
-----------------
| | |B| | | | | |
-----------------
The evaluation of the boards below is -3
-----------------
| | | | | | | | |
-----------------
| | |b| | | |b| |
-----------------
| |b| | | | | | |
-----------------
| | | | | | | | |
-----------------
| | | | | | | | |
-----------------
| | | | | | | | |
-----------------
| | | | | | | | |
-----------------
| | |B| | | |W| |
-----------------


KeyboardInterrupt: ignored

Gets stuckin an infinite loop but V3 in a much stronger position with 4 pieces left compared to V4's one. Now lets see when V3 goes first. 

In [20]:
#v4 vs v3
versusBots(state_of_playV4 ,state_of_playV3)

First Eval function is playing b
The evaluation of the boards below is 31.970000000000002
-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| |w| |w| |w| | |
-----------------
| | | | | | |w| |
-----------------
| | | | | | | | |
-----------------
|b| |b| |b| |b| |
-----------------
| |b| |b| |b| |b|
-----------------
|b| |b| |b| |b| |
-----------------
The evaluation of the boards below is 0
-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| |w| |w| |w| | |
-----------------
| | | | | | |w| |
-----------------
| | | | | |b| | |
-----------------
|b| |b| | | |b| |
-----------------
| |b| |b| |b| |b|
-----------------
|b| |b| |b| |b| |
-----------------
The evaluation of the boards below is 32.0
-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| |w| |w| |w| | |
-----------------
| | | | | | | | |
-----------------
| | | | | | | | |
-----------------
|b| |b| |w| |b| |

V3 beat V4 when it went first and was in a much stronger position when the game was interrupted when it went second. V3 therefore appears to be a stringer function than either V5 or v4 - indicating that using the row index as a multiplier is not improving the evaluation function. 

Using the row index assigned greater value to where the pieces on the board were in relation to being kinged. However, typically, the position on the board is relevant in terms of length and breath, because pieces closer to the centre of the board have more freedom and therefore more potental moves. The next evaluation function will try to incorportae this into it and see if it can best V3. 

In [24]:
def state_of_playV6(boardAsString):
    pieceValue = 1
    kingMultiplier = 2.5
#create values for each position on the board for whites and blacks, where the opposite side of th board to the one you start on, and the centre of the board, are more highly weigted. 
    white_position_value = [
        [4,  6,  8, 10, 10,  8,  6,  4],
        [5,  7,  9, 11, 11,  9,  7,  5],
        [6,  8, 10, 12, 12, 10,  8,  6],
        [7,  9, 11, 13, 13, 11,  9,  7],
        [8, 10, 12, 14, 14, 12, 10,  8],
        [9, 11, 13, 15, 15, 13, 11,  9],
        [10, 12, 14, 16, 16, 14, 12, 10],
        [11, 13, 15, 17, 17, 15, 13, 11]
    ]
    
    black_position_value = [
        [11, 13, 15, 17, 17, 15, 13, 11],
        [10, 12, 14, 16, 16, 14, 12, 10],
        [9, 11, 13, 15, 15, 13, 11,  9],
        [8, 10, 12, 14, 14, 12, 10,  8],
        [7,  9, 11, 13, 13, 11,  9,  7],
        [6,  8, 10, 12, 12, 10,  8,  6],
        [5,  7,  9, 11, 11,  9,  7,  5],
        [4,  6,  8, 10, 10,  8,  6,  4]
    ]
    
    whites = 0
    blacks = 0

    for row_idx, row in enumerate(boardAsString):
        for col_idx, piece in enumerate(row):
            if piece == 'w':
                whites += pieceValue + kingMultiplier * row.count('W') + white_position_value[row_idx][col_idx]/100
            elif piece == 'b':
                blacks += pieceValue + kingMultiplier * row.count('B') + black_position_value[row_idx][col_idx]/100
           
    evaluation = whites - blacks
    if whites == 0:
        return 'b', evaluation
    if blacks == 0:
        return 'w', evaluation
    return 'U', evaluation


In [25]:
#v3 vs v6
versusBots(state_of_playV3 ,state_of_playV6)

First Eval function is playing b
The evaluation of the boards below is 0
-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| | | |w| |w| |w|
-----------------
| | |w| | | | | |
-----------------
| | | | | | | | |
-----------------
|b| |b| |b| |b| |
-----------------
| |b| |b| |b| |b|
-----------------
|b| |b| |b| |b| |
-----------------
The evaluation of the boards below is 0.03999999999999737
-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| | | |w| |w| |w|
-----------------
| | |w| | | | | |
-----------------
| | | | | |b| | |
-----------------
|b| |b| |b| | | |
-----------------
| |b| |b| |b| |b|
-----------------
|b| |b| |b| |b| |
-----------------
The evaluation of the boards below is 0
-----------------
| |w| |w| |w| |w|
-----------------
| | |w| |w| |w| |
-----------------
| |w| |w| |w| |w|
-----------------
| | |w| | | | | |
-----------------
| | | | | |b| | |
-----------------
|b| |b| |b| | | |
-

KeyboardInterrupt: ignored

Gets stuck in an infinite loop but there are 4 V3 pieces and only 2 V6 pieces left indicating that V3 is the stronger function. Lets check now when V3 goes first. 

In [26]:
#v6 vs v3
versusBots(state_of_playV6 ,state_of_playV3)

First Eval function is playing b
The evaluation of the boards below is 0.009999999999999787
-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| |w| |w| |w| | |
-----------------
| | | | | | |w| |
-----------------
| | | | | | | | |
-----------------
|b| |b| |b| |b| |
-----------------
| |b| |b| |b| |b|
-----------------
|b| |b| |b| |b| |
-----------------
The evaluation of the boards below is 0
-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| |w| |w| |w| | |
-----------------
| | | | | | |w| |
-----------------
| | | | | |b| | |
-----------------
|b| |b| | | |b| |
-----------------
| |b| |b| |b| |b|
-----------------
|b| |b| |b| |b| |
-----------------
The evaluation of the boards below is 1.1099999999999977
-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| |w| |w| |w| | |
-----------------
| | | | | | | | |
-----------------
| | | | | | | | |
-----------------
|

KeyboardInterrupt: ignored

Gets stuck in an infinite loop again but V6 is in a slightly stronger position with 3 pieces (2 kings and 1 normal) compared to V3 (1 king and 2 normal). In conclusion, V6 looked stronger when V3 went first, and V3 looked stronger when V6 went first. I am going to adjust the strength of the king multiplier for v7 (still using the v6) positioning system and see if that can trup v3. 

In [27]:
def state_of_playV7(boardAsString):
    pieceValue = 2
    kingMultiplier = 3
#create values for each position on the board for whites and blacks, where the opposite side of th board to the one you start on, and the centre of the board, are more highly weigted. 
    white_position_value = [
        [4,  6,  8, 10, 10,  8,  6,  4],
        [5,  7,  9, 11, 11,  9,  7,  5],
        [6,  8, 10, 12, 12, 10,  8,  6],
        [7,  9, 11, 13, 13, 11,  9,  7],
        [8, 10, 12, 14, 14, 12, 10,  8],
        [9, 11, 13, 15, 15, 13, 11,  9],
        [10, 12, 14, 16, 16, 14, 12, 10],
        [11, 13, 15, 17, 17, 15, 13, 11]
    ]
    
    black_position_value = [
        [11, 13, 15, 17, 17, 15, 13, 11],
        [10, 12, 14, 16, 16, 14, 12, 10],
        [9, 11, 13, 15, 15, 13, 11,  9],
        [8, 10, 12, 14, 14, 12, 10,  8],
        [7,  9, 11, 13, 13, 11,  9,  7],
        [6,  8, 10, 12, 12, 10,  8,  6],
        [5,  7,  9, 11, 11,  9,  7,  5],
        [4,  6,  8, 10, 10,  8,  6,  4]
    ]
    
    whites = 0
    blacks = 0

    for row_idx, row in enumerate(boardAsString):
        for col_idx, piece in enumerate(row):
            if piece == 'w':
                whites += pieceValue + kingMultiplier * row.count('W') + white_position_value[row_idx][col_idx]/100
            elif piece == 'b':
                blacks += pieceValue + kingMultiplier * row.count('B') + black_position_value[row_idx][col_idx]/100
           
    evaluation = whites - blacks
    if whites == 0:
        return 'b', evaluation
    if blacks == 0:
        return 'w', evaluation
    return 'U', evaluation


In [28]:
#v7 vs v3
versusBots(state_of_playV7 ,state_of_playV3)

First Eval function is playing b
The evaluation of the boards below is 0.009999999999994458
-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| |w| |w| |w| | |
-----------------
| | | | | | |w| |
-----------------
| | | | | | | | |
-----------------
|b| |b| |b| |b| |
-----------------
| |b| |b| |b| |b|
-----------------
|b| |b| |b| |b| |
-----------------
The evaluation of the boards below is 0
-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| |w| |w| |w| | |
-----------------
| | | | | | |w| |
-----------------
| | | | | |b| | |
-----------------
|b| |b| | | |b| |
-----------------
| |b| |b| |b| |b|
-----------------
|b| |b| |b| |b| |
-----------------
The evaluation of the boards below is 2.109999999999996
-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| |w| |w| |w| | |
-----------------
| | | | | | | | |
-----------------
| | | | | | | | |
-----------------
|b

In [29]:
#v3 vs v7
versusBots(state_of_playV3 ,state_of_playV7)

First Eval function is playing b
The evaluation of the boards below is 0
-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| | | |w| |w| |w|
-----------------
| | |w| | | | | |
-----------------
| | | | | | | | |
-----------------
|b| |b| |b| |b| |
-----------------
| |b| |b| |b| |b|
-----------------
|b| |b| |b| |b| |
-----------------
The evaluation of the boards below is 0.04000000000000625
-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| | | |w| |w| |w|
-----------------
| | |w| | | | | |
-----------------
| | | | | |b| | |
-----------------
|b| |b| |b| | | |
-----------------
| |b| |b| |b| |b|
-----------------
|b| |b| |b| |b| |
-----------------
The evaluation of the boards below is 0
-----------------
| |w| |w| |w| |w|
-----------------
| | |w| |w| |w| |
-----------------
| |w| |w| |w| |w|
-----------------
| | |w| | | | | |
-----------------
| | | | | |b| | |
-----------------
|b| |b| |b| | | |
-

KeyboardInterrupt: ignored

**V7 vs V3**

When V3 went first it beat V7 and when V7 went first it ended in an infinite loop but V3 had the stronger board poisition (4 pieces - 3 kings and 1 normal vs 3 pieces - 1 king and 2 normal). V3 therefore seems to outperform V7. As V3 beat V7 more conclusively than it did V6, it appears lowering the king value made the function weaker. Lets now increase in v8 and see what happens.

In [34]:
def state_of_playV8(boardAsString):
    pieceValue = 1
    kingMultiplier = 10
#create values for each position on the board for whites and blacks, where the opposite side of th board to the one you start on, and the centre of the board, are more highly weigted. 
    white_position_value = [
        [4,  6,  8, 10, 10,  8,  6,  4],
        [5,  7,  9, 11, 11,  9,  7,  5],
        [6,  8, 10, 12, 12, 10,  8,  6],
        [7,  9, 11, 13, 13, 11,  9,  7],
        [8, 10, 12, 14, 14, 12, 10,  8],
        [9, 11, 13, 15, 15, 13, 11,  9],
        [10, 12, 14, 16, 16, 14, 12, 10],
        [11, 13, 15, 17, 17, 15, 13, 11]
    ]
    
    black_position_value = [
        [11, 13, 15, 17, 17, 15, 13, 11],
        [10, 12, 14, 16, 16, 14, 12, 10],
        [9, 11, 13, 15, 15, 13, 11,  9],
        [8, 10, 12, 14, 14, 12, 10,  8],
        [7,  9, 11, 13, 13, 11,  9,  7],
        [6,  8, 10, 12, 12, 10,  8,  6],
        [5,  7,  9, 11, 11,  9,  7,  5],
        [4,  6,  8, 10, 10,  8,  6,  4]
    ]
    
    whites = 0
    blacks = 0

    for row_idx, row in enumerate(boardAsString):
        for col_idx, piece in enumerate(row):
            if piece == 'w':
                whites += pieceValue + kingMultiplier * row.count('W') + white_position_value[row_idx][col_idx]/100
            elif piece == 'b':
                blacks += pieceValue + kingMultiplier * row.count('B') + black_position_value[row_idx][col_idx]/100
           
    evaluation = whites - blacks
    if whites == 0:
        return 'b', evaluation
    if blacks == 0:
        return 'w', evaluation
    return 'U', evaluation


#v3 vs v7
# versusBots(state_of_playV3 ,state_of_playV7)

In [32]:
#v3 vs v8
versusBots(state_of_playV3 ,state_of_playV8)

First Eval function is playing b
The evaluation of the boards below is 0
-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| | | |w| |w| |w|
-----------------
| | |w| | | | | |
-----------------
| | | | | | | | |
-----------------
|b| |b| |b| |b| |
-----------------
| |b| |b| |b| |b|
-----------------
|b| |b| |b| |b| |
-----------------
The evaluation of the boards below is 0.03999999999999737
-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| | | |w| |w| |w|
-----------------
| | |w| | | | | |
-----------------
| | | | | |b| | |
-----------------
|b| |b| |b| | | |
-----------------
| |b| |b| |b| |b|
-----------------
|b| |b| |b| |b| |
-----------------
The evaluation of the boards below is 0
-----------------
| |w| |w| |w| |w|
-----------------
| | |w| |w| |w| |
-----------------
| |w| |w| |w| |w|
-----------------
| | |w| | | | | |
-----------------
| | | | | |b| | |
-----------------
|b| |b| |b| | | |
-

KeyboardInterrupt: ignored

Stuck in an infinite loop but V3 in the stronger position.

In [33]:
#v8 vs v3
versusBots(state_of_playV8 ,state_of_playV3)

First Eval function is playing b
The evaluation of the boards below is 0.009999999999999787
-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| |w| |w| |w| | |
-----------------
| | | | | | |w| |
-----------------
| | | | | | | | |
-----------------
|b| |b| |b| |b| |
-----------------
| |b| |b| |b| |b|
-----------------
|b| |b| |b| |b| |
-----------------
The evaluation of the boards below is 0
-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| |w| |w| |w| | |
-----------------
| | | | | | |w| |
-----------------
| | | | | |b| | |
-----------------
|b| |b| | | |b| |
-----------------
| |b| |b| |b| |b|
-----------------
|b| |b| |b| |b| |
-----------------
The evaluation of the boards below is 1.1099999999999977
-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| |w| |w| |w| | |
-----------------
| | | | | | | | |
-----------------
| | | | | | | | |
-----------------
|

KeyboardInterrupt: ignored

**V3 vs V8**
Stuck in an infinite loop again but V3 is in a slightly strongr position as it has one more king, even though they have the same number of pieces. V3 is therefore stronger when it goes first or second. None of the functions considering for board position are outperforming the simple V3 function - maybe the board position affects are too subtle? Lets make v9 have stronger board effects  

In [38]:
def state_of_playV9(boardAsString):
    pieceValue = 1
    kingMultiplier = 2.5
#create values for each position on the board for whites and blacks, where the opposite side of th board to the one you start on, and the centre of the board, are more highly weigted. 
    white_position_value = [
        [4,  6,  8, 10, 10,  8,  6,  4],
        [5,  7,  9, 11, 11,  9,  7,  5],
        [6,  8, 10, 12, 12, 10,  8,  6],
        [7,  9, 11, 13, 13, 11,  9,  7],
        [8, 10, 12, 14, 14, 12, 10,  8],
        [9, 11, 13, 15, 15, 13, 11,  9],
        [10, 12, 14, 16, 16, 14, 12, 10],
        [11, 13, 15, 17, 17, 15, 13, 11]
    ]
    
    black_position_value = [
        [11, 13, 15, 17, 17, 15, 13, 11],
        [10, 12, 14, 16, 16, 14, 12, 10],
        [9, 11, 13, 15, 15, 13, 11,  9],
        [8, 10, 12, 14, 14, 12, 10,  8],
        [7,  9, 11, 13, 13, 11,  9,  7],
        [6,  8, 10, 12, 12, 10,  8,  6],
        [5,  7,  9, 11, 11,  9,  7,  5],
        [4,  6,  8, 10, 10,  8,  6,  4]
    ]
    
    whites = 0
    blacks = 0

#divide by 10 rather than 100 to increase the effect of board position
    for row_idx, row in enumerate(boardAsString):
        for col_idx, piece in enumerate(row):
            if piece == 'w':
                whites += pieceValue + kingMultiplier * row.count('W') + white_position_value[row_idx][col_idx]/10
            elif piece == 'b':
                blacks += pieceValue + kingMultiplier * row.count('B') + black_position_value[row_idx][col_idx]/10
           
    evaluation = whites - blacks
    if whites == 0:
        return 'b', evaluation
    if blacks == 0:
        return 'w', evaluation
    return 'U', evaluation


#v3 vs v7
# versusBots(state_of_playV3 ,state_of_playV7)

In [39]:
#v3 vs v9
versusBots(state_of_playV3 ,state_of_playV9)

First Eval function is playing b
The evaluation of the boards below is 0
-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| | | |w| |w| |w|
-----------------
| | |w| | | | | |
-----------------
| | | | | | | | |
-----------------
|b| |b| |b| |b| |
-----------------
| |b| |b| |b| |b|
-----------------
|b| |b| |b| |b| |
-----------------
The evaluation of the boards below is 0.3999999999999986
-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| | | |w| |w| |w|
-----------------
| | |w| | | | | |
-----------------
| | | | | |b| | |
-----------------
|b| |b| |b| | | |
-----------------
| |b| |b| |b| |b|
-----------------
|b| |b| |b| |b| |
-----------------
The evaluation of the boards below is 0
-----------------
| |w| |w| |w| |w|
-----------------
| | |w| |w| |w| |
-----------------
| |w| |w| |w| |w|
-----------------
| | |w| | | | | |
-----------------
| | | | | |b| | |
-----------------
|b| |b| |b| | | |
--

KeyboardInterrupt: ignored

In [40]:
#v9 vs v3
versusBots(state_of_playV9 ,state_of_playV3)

First Eval function is playing b
The evaluation of the boards below is 0.09999999999999787
-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| |w| |w| |w| | |
-----------------
| | | | | | |w| |
-----------------
| | | | | | | | |
-----------------
|b| |b| |b| |b| |
-----------------
| |b| |b| |b| |b|
-----------------
|b| |b| |b| |b| |
-----------------
The evaluation of the boards below is 0
-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| |w| |w| |w| | |
-----------------
| | | | | | |w| |
-----------------
| | | | | |b| | |
-----------------
|b| |b| |b| | | |
-----------------
| |b| |b| |b| |b|
-----------------
|b| |b| |b| |b| |
-----------------
The evaluation of the boards below is 0.29999999999999716
-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| |w| |w| |w| | |
-----------------
| | | | | | | | |
-----------------
| | | | | |b| |w|
-----------------
|

**V3 vs V9**
When V9 went first the match resulted in an infinite loop, but V3 again had the stringer pieces left on the board (3 kings and one normal), compared to V9's 4 pieces (1 king and 3 normal). 

When V3 went first it ended up winning - so once again V3 is the strongest function. 

In [60]:
def state_of_playV10(boardAsString):
    pieceValue = 1
    kingMultiplier = 2.5
#create values for each position on the board for whites and blacks, where the opposite side of th board to the one you start on, and the centre of the board, are more highly weigted. 

    white_position_value = [
        [4,  6,  8, 10, 10,  8,  6,  4],
        [5,  7,  9, 11, 11,  9,  7,  5],
        [6,  8, 10, 12, 12, 10,  8,  6],
        [7,  9, 11, 13, 13, 11,  9,  7],
        [8, 10, 12, 14, 14, 12, 10,  8],
        [9, 11, 13, 15, 15, 13, 11,  9],
        [10, 12, 14, 16, 16, 14, 12, 10],
        [11, 13, 15, 17, 17, 15, 13, 11]
    ]
    
    black_position_value = [
        [11, 13, 15, 17, 17, 15, 13, 11],
        [10, 12, 14, 16, 16, 14, 12, 10],
        [9, 11, 13, 15, 15, 13, 11,  9],
        [8, 10, 12, 14, 14, 12, 10,  8],
        [7,  9, 11, 13, 13, 11,  9,  7],
        [6,  8, 10, 12, 12, 10,  8,  6],
        [5,  7,  9, 11, 11,  9,  7,  5],
        [4,  6,  8, 10, 10,  8,  6,  4]
    ]
    
    white_king_position_value = [
     [4,  6,  8, 10, 10,  8,  6,  4],
	  	[4,  6,  8, 10, 10,  8,  6,  4],
      [4,  6,  8, 10, 10,  8,  6,  4],
	   	[4,  6,  8, 10, 10,  8,  6,  4],
      [4,  6,  8, 10, 10,  8,  6,  4],
      [4,  6,  8, 10, 10,  8,  6,  4],
      [4,  6,  8, 10, 10,  8,  6,  4],
      [4,  6,  8, 10, 10,  8,  6,  4]
    ]
	
    black_king_position_value = [
      [4,  6,  8, 10, 10,  8,  6,  4],
	  	[4,  6,  8, 10, 10,  8,  6,  4],
      [4,  6,  8, 10, 10,  8,  6,  4],
	   	[4,  6,  8, 10, 10,  8,  6,  4],
      [4,  6,  8, 10, 10,  8,  6,  4],
      [4,  6,  8, 10, 10,  8,  6,  4],
      [4,  6,  8, 10, 10,  8,  6,  4],
      [4,  6,  8, 10, 10,  8,  6,  4]
    ]
	
	
	
    whites = 0
    blacks = 0


    for row_idx, row in enumerate(boardAsString):
        for col_idx, piece in enumerate(row):
            if piece == 'w':
                whites += pieceValue + kingMultiplier * row.count('W') + white_position_value[row_idx][col_idx]/100
            elif piece == 'W':
                whites += pieceValue + kingMultiplier * row.count('W') + white_king_position_value[row_idx][col_idx]/100
            elif piece == 'b':
                blacks += pieceValue + kingMultiplier * row.count('B') + black_position_value[row_idx][col_idx]/100
            elif piece == 'B':
                blacks += pieceValue + kingMultiplier * row.count('B') + black_king_position_value[row_idx][col_idx]/100
           
    evaluation = whites - blacks
    if whites == 0:
        return 'b', evaluation
    if blacks == 0:
        return 'w', evaluation
    return 'U', evaluation


#v3 vs v7
# versusBots(state_of_playV3 ,state_of_playV7)

In [61]:
#v3 vs v10
versusBots(state_of_playV3 ,state_of_playV10)

First Eval function is playing b
The evaluation of the boards below is 0
-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| | | |w| |w| |w|
-----------------
| | |w| | | | | |
-----------------
| | | | | | | | |
-----------------
|b| |b| |b| |b| |
-----------------
| |b| |b| |b| |b|
-----------------
|b| |b| |b| |b| |
-----------------
The evaluation of the boards below is 0.03999999999999737
-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| | | |w| |w| |w|
-----------------
| | |w| | | | | |
-----------------
| | | | | |b| | |
-----------------
|b| |b| |b| | | |
-----------------
| |b| |b| |b| |b|
-----------------
|b| |b| |b| |b| |
-----------------
The evaluation of the boards below is 0
-----------------
| |w| |w| |w| |w|
-----------------
| | |w| |w| |w| |
-----------------
| |w| |w| |w| |w|
-----------------
| | |w| | | | | |
-----------------
| | | | | |b| | |
-----------------
|b| |b| |b| | | |
-

KeyboardInterrupt: ignored

In [62]:
#v10 vs v3
versusBots(state_of_playV10 ,state_of_playV3)

First Eval function is playing b
The evaluation of the boards below is 0.009999999999999787
-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| |w| |w| |w| | |
-----------------
| | | | | | |w| |
-----------------
| | | | | | | | |
-----------------
|b| |b| |b| |b| |
-----------------
| |b| |b| |b| |b|
-----------------
|b| |b| |b| |b| |
-----------------
The evaluation of the boards below is 0
-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| |w| |w| |w| | |
-----------------
| | | | | | |w| |
-----------------
| | | | | |b| | |
-----------------
|b| |b| | | |b| |
-----------------
| |b| |b| |b| |b|
-----------------
|b| |b| |b| |b| |
-----------------
The evaluation of the boards below is 1.1099999999999977
-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| |w| |w| |w| | |
-----------------
| | | | | | | | |
-----------------
| | | | | | | | |
-----------------
|

KeyboardInterrupt: ignored

**V3 vs V10:**
When V10 went first, both teams ended up with two kings left on the board stuck in a loop - a draw. 

When V3 went first, both colours ended with three pieces stuck in a loop (1 king and 2 normal) but V10 had more advanced pieces closer to becoming kinged.V10 appears therefore to be at least as good as V3. Lets test V10 against some of the other agents. 


In [63]:
#v10 vs v7
versusBots(state_of_playV10 ,state_of_playV7)

First Eval function is playing b
The evaluation of the boards below is 0.02999999999999936
-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| | | |w| |w| |w|
-----------------
| | |w| | | | | |
-----------------
| | | | | | | | |
-----------------
|b| |b| |b| |b| |
-----------------
| |b| |b| |b| |b|
-----------------
|b| |b| |b| |b| |
-----------------
The evaluation of the boards below is 0.03999999999999915
-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| | | |w| |w| |w|
-----------------
| | |w| | | | | |
-----------------
| |b| | | | | | |
-----------------
|b| | | |b| |b| |
-----------------
| |b| |b| |b| |b|
-----------------
|b| |b| |b| |b| |
-----------------
The evaluation of the boards below is 0.049999999999998934
-----------------
| |w| |w| |w| |w|
-----------------
| | |w| |w| |w| |
-----------------
| |w| |w| |w| |w|
-----------------
| | |w| | | | | |
-----------------
| |b| | | | | | |


In [65]:
#v10 vs v7
versusBots(state_of_playV7 ,state_of_playV10)

First Eval function is playing b
The evaluation of the boards below is 0.030000000000001137
-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| | | |w| |w| |w|
-----------------
| | |w| | | | | |
-----------------
| | | | | | | | |
-----------------
|b| |b| |b| |b| |
-----------------
| |b| |b| |b| |b|
-----------------
|b| |b| |b| |b| |
-----------------
The evaluation of the boards below is 0.03999999999999737
-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| | | |w| |w| |w|
-----------------
| | |w| | | | | |
-----------------
| |b| | | | | | |
-----------------
|b| | | |b| |b| |
-----------------
| |b| |b| |b| |b|
-----------------
|b| |b| |b| |b| |
-----------------
The evaluation of the boards below is 0.06999999999999673
-----------------
| |w| |w| |w| |w|
-----------------
|w| |w| |w| |w| |
-----------------
| | | | | |w| |w|
-----------------
| | |w| |w| | | |
-----------------
| |b| | | | | | |


KeyboardInterrupt: ignored

**V10 vs V7**

When V7 went first V10 won and when V10 went first it was stuck in a loop, with V10 having two kings and one normal piece and V7 having one king and two normal pieces. Overall V10 is the better function. 

V10 - which takes into account the position on the board in terms of centre and how close normal pieces are to be kinged, appears to be the best evaluation function. 